<a href="https://colab.research.google.com/github/sadaha/Coursera_Capstone.-/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Here I will write my code for the Segmenting and Clustering Neighborhoods in Toronto project of the IBM Data science course.**


## Question 1

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
URL = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
page = requests.get(URL)

soup = bs(page.content, 'html.parser')

Here I have imported the raw data from the wiki page. And will change it now into a dataframe.

In [ ]:
wiki_data = pd.read_html(page.text)
wiki_data

This reads three lists objects from the web page data, one of which is the table we want

In [ ]:
len(wiki_data), type(wiki_data)

(3, list)

The first list object is the data wanted. So remove the others

In [ ]:
toronto_data = wiki_data[0]
toronto_data

Remove the rows with boroughs that are not assigned

In [ ]:
df = toronto_data[toronto_data["Borough"] != "Not assigned"]
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [ ]:
df.shape

(103, 3)

## Question 2

First I need to get the location data for the post codes

In [ ]:
latlon_data = pd.read_csv("https://cocl.us/Geospatial_data")
latlon_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [ ]:
df = df.join(latlon_data.set_index('Postal Code'), on='Postal Code', how='inner')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


I have joined the geospatial data with the webscraped dataset.

#Question 3

In [ ]:
from geopy.geocoders import Nominatim
import folium

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

The coordinates of Toronto are 43.6534817, -79.3839347.


In [ ]:
# Creating the map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_Toronto)  
    
map_Toronto

In [ ]:
CLIENT_ID = 'LDIJF4KI5VGMMA3NNDLFZWHR12TCMNTUL0TUC3QPZ3SJD040' 
CLIENT_SECRET = 'EOOOZ3EF5N0FOMNUJVTDV0SXVUVVEBMWPFXMNBK1R5K4H55A'
VERSION = '20180605'

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
venues_in_toronto = getNearbyVenues(df['Neighbourhood'], df['Latitude'], df['Longitude'])

In [ ]:
toronto_venue_cat = pd.get_dummies(venues_in_toronto[['Venue Category']], prefix="", prefix_sep="")
toronto_venue_cat

I had difficulty getting my foursquare account to work, so I stopped the project here and hope this'll be enough to pass.